*Datasets used in this example are a system of hard hexagons, simulated in the NVT thermodynamic ensemble in HOOMD-Blue, for a dense fluid (phi065) and a solid (phi075)*

In [31]:
from bokeh.io import output_notebook
output_notebook()
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import gridplot
import numpy as np
import time
import PIL.Image
import io
import IPython.display
from freud import parallel
parallel.setNumThreads(4)

# helper functions used in the notebook are below; you are free to disregard

def showarray(a, fmt='png'):
    """
    uses PIL to display an image rendered externally.
    
    Currently not used
    """
    f = io.BytesIO()
    PIL.Image.fromarray(a, mode='RGBA').save(f, fmt)
#     PIL.Image.fromarray(a, mode='RGBA').save("out.png")
    return IPython.display.display(IPython.display.Image(data=f.getvalue(), width=600))

def default_bokeh(p):
    """
    wrapper which takes the default bokeh outputs and changes them to more sensible values
    """
    p.title.text_font_size = "18pt"
    p.title.align = "center"

    p.xaxis.axis_label_text_font_size = "14pt"
    p.yaxis.axis_label_text_font_size = "14pt"

    p.xaxis.major_tick_in = 10
    p.xaxis.major_tick_out = 0
    p.xaxis.minor_tick_in = 5
    p.xaxis.minor_tick_out = 0

    p.yaxis.major_tick_in = 10
    p.yaxis.major_tick_out = 0
    p.yaxis.minor_tick_in = 5
    p.yaxis.minor_tick_out = 0

    p.xaxis.major_label_text_font_size = "12pt"
    p.yaxis.major_label_text_font_size = "12pt"

def cubeellipse(theta, lam=0.5, gamma=1., s=4.0, r=1., h=1.):
    """Create an RGB colormap from an input angle theta. Takes lam (a list of
    intensity values, from 0 to 1), gamma (a nonlinear weighting power),
    s (starting angle), r (number of revolutions around the circle), and
    h (a hue factor)."""
    import numpy
    lam = lam**gamma

    a = h*lam*(1 - lam)*.5
    v = numpy.array([[-.14861, 1.78277], [-.29227, -.90649], [1.97294, 0.]], dtype=numpy.float32)
    ctarray = numpy.array([numpy.cos(theta*r + s), numpy.sin(theta*r + s)], dtype=numpy.float32)
    # convert to 255 rgb
    ctarray = (lam + a*v.dot(ctarray)).T
    ctarray *= 255
    ctarray = ctarray.astype(dtype=np.int32)
    return ctarray

def local_to_global(verts, positions, orientations):
    """
    Take a list of vertices, positions, and orientations and create
    a list of vertices in the "global coordinate system" for plotting
    in bokeh
    """
    num_particles = len(positions)
    num_verts = len(verts)
    # create list of vertices in the "local reference frame" i.e.
    # centered at (0,0)
    l_verts = np.zeros(shape=(num_particles, num_verts, 2), dtype=np.float32)
    l_verts[:] = verts
    # create array of rotation matrices
    rot_mat = np.zeros(shape=(num_particles, 2, 2), dtype=np.float32)
    for i, theta in enumerate(orientations):
        rot_mat[i] = [[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]]
    # rotate; uses einsum for speed; please see numpy documentation
    # for more information
    r_verts = np.einsum("lij,lkj->lki", rot_mat, l_verts)
    # now translate to global coordinates
    # need to create a position array with same shape as vertex array
    l_pos = np.zeros(shape=(num_particles, num_verts, 2), dtype=np.float32)
    for i in range(num_particles):
        for j in range(len(verts)):
            l_pos[i,j] = positions[i]
    # translate
    output_array = np.add(r_verts, l_pos)
    return output_array

def clamp(x):
    """
    limit values between 0 and 255
    http://stackoverflow.com/questions/3380726/converting-a-rgb-color-tuple-to-a-six-digit-code-in-python
    """
    return max(0, min(x, 255))

Loading BokehJS ...

# Hexatic Order Parameter

The hexatic order parameter measures how closely the local environment around a particle resembles perfect $k$-atic symmetry e.g. how closely the environment resembles hexagonal/hexatic symmetry if the order parameter is $k=6$. The order parameter is given by:

$$
\psi_k \left( i \right) = \frac{1}{n} \sum \limits_j^n e^{k i \phi_{ij}}
$$

where $\phi_{ij}$ is the angle between the vector $r_{ij}$ and $ \vec{\left(1, 0\right)}$

The pseudocode is given below

~~~
for each particle i:
    neighbors = nearestNeighbors(i, n):
    for each particle j in neighbors:
        r_ij = position[j] - position[i]
        psi_ij = arctan2(r_ij.y, r_ij.x)
        psi_array[i] += exp(complex(0,k*psi_ij))
~~~

An example which calculates and renders the scene for $\phi=0.65$ is shown below:

In [35]:
from freud import box, order
# create hexatic object
hex_order = order.HexOrderParameter(rmax=1.2, k=6, n=6);

# load the data
data_path = "ex_data/phi065"
box_data = np.load("{}/box_data.npy".format(data_path))
pos_data = np.load("{}/pos_data.npy".format(data_path))
quat_data = np.load("{}/quat_data.npy".format(data_path))
n_frames = pos_data.shape[0]

# grab data from last frame
l_box = box_data[-1]
l_pos = pos_data[-1]
l_quat = quat_data[-1]
l_ang = 2*np.arctan2(np.copy(l_quat[:,3]), np.copy(l_quat[:,0]))

# create box
fbox = box.Box(Lx=l_box["Lx"], Ly=l_box["Ly"], is2D=True)

# compute hexatic order for 6 nearest neighbors
start_time = time.time()
hex_order.compute(fbox, l_pos)
stop_time = time.time()
print("time to calc 1 frame = {}".format(stop_time-start_time))
# get values from freud object
psi_k = hex_order.getPsi()
avg_psi_k = np.mean(psi_k)

# render in bokeh
# vertex positions for hexagons
verts = [[0.537284965911771, 0.31020161970069976],
         [3.7988742065678664e-17, 0.6204032394013997],
         [-0.5372849659117709, 0.31020161970070004],
         [-0.5372849659117711, -0.31020161970069976],
         [-1.1396622619703597e-16, -0.6204032394013997],
         [0.5372849659117711, -0.3102016197006997]]
verts = np.array(verts)
# create array of transformed positions
patches = local_to_global(verts, l_pos[:,0:2], l_ang)
# create an array of angles relative to the average
a = np.angle(psi_k) - np.angle(avg_psi_k)
# turn into an rgb array of tuples
color = [tuple(cubeellipse(x)) for x in a]
# bokeh (as of this version) requires hex colors, so convert rgb to hex
hex_color = ["#{0:02x}{1:02x}{2:02x}".format(clamp(r), clamp(g), clamp(b)) for (r,g,b) in color]
# plot
p = figure(title="Hexatic Order Parameter visualization")
p.patches(xs=patches[:,:,0].tolist(), ys=patches[:,:,1].tolist(),
    fill_color=hex_color, line_color="black")
default_bokeh(p)
show(p)

time to calc 1 frame = 0.03148293495178223


An example which shows $\phi=0.75$ is shown below

In [34]:
from freud import box, order
# create hexatic object
hex_order = order.HexOrderParameter(rmax=1.2, k=6, n=6);

# load the data
data_path = "ex_data/phi075"
box_data = np.load("{}/box_data.npy".format(data_path))
pos_data = np.load("{}/pos_data.npy".format(data_path))
quat_data = np.load("{}/quat_data.npy".format(data_path))
n_frames = pos_data.shape[0]

# grab data from last frame
l_box = box_data[-1]
l_pos = pos_data[-1]
l_quat = quat_data[-1]
l_ang = 2*np.arctan2(np.copy(l_quat[:,3]), np.copy(l_quat[:,0]))

# create box
fbox = box.Box(Lx=l_box["Lx"], Ly=l_box["Ly"], is2D=True)

# compute hexatic order for 6 nearest neighbors
start_time = time.time()
hex_order.compute(fbox, l_pos)
stop_time = time.time()
print("time to calc 1 frame = {}".format(stop_time-start_time))
# get values from freud object
psi_k = hex_order.getPsi()
avg_psi_k = np.mean(psi_k)

# render in bokeh
# vertex positions for hexagons
verts = [[0.537284965911771, 0.31020161970069976],
         [3.7988742065678664e-17, 0.6204032394013997],
         [-0.5372849659117709, 0.31020161970070004],
         [-0.5372849659117711, -0.31020161970069976],
         [-1.1396622619703597e-16, -0.6204032394013997],
         [0.5372849659117711, -0.3102016197006997]]
verts = np.array(verts)
# create array of transformed positions
patches = local_to_global(verts, l_pos[:,0:2], l_ang)
# create an array of angles relative to the average
a = np.angle(psi_k) - np.angle(avg_psi_k)
# turn into an rgb array of tuples
color = [tuple(cubeellipse(x)) for x in a]
# bokeh (as of this version) requires hex colors, so convert rgb to hex
hex_color = ["#{0:02x}{1:02x}{2:02x}".format(clamp(r), clamp(g), clamp(b)) for (r,g,b) in color]
# plot
p = figure(title="Hexatic Order Parameter visualization")
p.patches(xs=patches[:,:,0].tolist(), ys=patches[:,:,1].tolist(),
    fill_color=hex_color, line_color="black")
default_bokeh(p)
show(p)

time to calc 1 frame = 0.03270721435546875
